In [ ]:
# Only on stage and prod
!pip install Levenshtein==0.17.0
!pip install zeno_etl_libs_v3==1.0.1
!pip install openpyxl==3.0.10

In [21]:
"""
Purpose: Fuzzy match to get patient names and store these patients in table,
and finally email the data on regular basis.

author : neha.karekar@zeno.health
"""

'\nPurpose: Fuzzy match to get patient names and store these patients in table,\nand finally email the data on regular basis.\n\nauthor : neha.karekar@zeno.health\n'

In [20]:
import sys
import os
import pandas as pd
import datetime
from dateutil.tz import gettz
import numpy as np
import Levenshtein as lev

In [3]:
sys.path.append('../../../..')

In [ ]:
from zeno_etl_libs.logger import get_logger
from zeno_etl_libs.db.db import DB
from zeno_etl_libs.helper.aws.s3 import S3
from zeno_etl_libs.helper import helper
from zeno_etl_libs.helper.email.email import Email

## Pass Params


In [1]:
env = "dev"
full_run = 0
email_to ="NA"
batch_size = 10

In [6]:
os.environ['env'] = env
logger = get_logger()

In [7]:
logger.info(f"env: {env}")
logger.info(f"full_run: {full_run}")
logger.info(f"batch_size: {batch_size}")
logger.info(f"email_to: {email_to}")

2022-07-20 17:14:01,857 - root - INFO - env: dev
2022-07-20 17:14:01,859 - root - INFO - full_run: 0
2022-07-20 17:14:01,861 - root - INFO - batch_size: 10
2022-07-20 17:14:01,862 - root - INFO - email_to: NA


In [9]:
# read
rs_db = DB()
rs_db.open_connection()

# write
rs_db_write = DB(read_only=False)
rs_db_write.open_connection()

s3 = S3()

In [10]:
schema = 'prod2-generico'
patients_table = 'playstore-patients'
reviews_table = 'playstore-reviews'
patients_table_info = helper.get_table_info(db=rs_db_write, table_name=patients_table, schema=schema)

In [11]:
def get_last_processed_review_date():
    # last review date in "playstore-patients" table
    query = f""" select max("review-created-at") last_review_at from "{schema}"."{patients_table}" """
    df = rs_db_write.get_df(query=query)
    df['last_review_at'].fillna(np.nan, inplace=True)
    last_review_at = df['last_review_at'].to_string(index=False)
    logger.info(f"last_review_at in play store patients table: {last_review_at}")
    return last_review_at

In [12]:
def get_orders(start_datetime, end_datetime):
    #Fetch zeno orders for a given date range
    zeno_q = f"""
        select
            zo.id as "zeno-order-id-before-review" ,
            zo."patient-id" ,
            zo."created-at" as "order-created-at",
            p.phone,
            p."name" as "matched-name"
        from
            "prod2-generico"."zeno-order" zo
        left join "prod2-generico".patients p on
            zo."patient-id" = p.id
        where
            zo."created-at" > '{start_datetime}'
            and zo."created-at" <= '{end_datetime}'
            and p."name" is not null
        """
    df = rs_db.get_df(zeno_q)
    return df

In [13]:
review_filter = ""
if full_run:
    """ No filter, take all """
    """ and truncate the playstore-patients tables """
    logger.info(f"Full run is: {full_run}, so truncating the table.")
    query = f""" truncate table "{schema}"."{patients_table}"; """
    rs_db_write.execute(query=query)
else:
    last_processed_review_at = get_last_processed_review_date()
    if last_processed_review_at == 'NaN':
        """ No filter, take all """
        pass
    else:
        review_filter = f""" and "review-created-at" > '{last_processed_review_at}' """

2022-07-20 17:14:09,147 - root - INFO - last_review_at in play store patients table: 2021-02-09 06:45:47


In [14]:
review_filter

' and "review-created-at" > \'2021-02-09 06:45:47\' '

In [15]:
# Fetching required reviews from playstore-review table
query = f"""
    select
        "review-id",
        "review",
        "author-name",
        "review-created-at",
        "star-rating"
    from
        "{schema}"."{reviews_table}"
    where
        "review-id" != ''
        {review_filter}
    order by 
        "review-created-at" asc
"""
reviews_df = rs_db.get_df(query=query)
reviews_df['review-created-at'] = pd.to_datetime(reviews_df['review-created-at'])

In [16]:
if reviews_df.empty:
    logger.info("No reviews to process, to stopping here.")

    rs_db.close_connection()
    rs_db_write.close_connection()

    exit()
else:
    logger.info(f"Total reviews to be processed: {len(reviews_df)}")

2022-07-20 17:15:44,242 - root - INFO - Total reviews to be processed: 776


In [2]:
# Process review in batches
counter = 1
for b_df in helper.batch(reviews_df, batch_size):
    print(f"Counter: {counter}, len: {len(b_df)}\n")

    # Do all the processing
    logger.info(f" b_df {counter}: {b_df.head(1).transpose()} \n\n")

    start_datetime = b_df['review-created-at'].min() - datetime.timedelta(days=7)
    end_datetime = b_df['review-created-at'].max()
    orders_df = get_orders(start_datetime=start_datetime, end_datetime=end_datetime)

    # Cross join reviews and orders
    b_df['i'] = 1
    orders_df['i'] = 1
    df = pd.merge(b_df, orders_df, how='inner', on='i')
    df['author-name'] = df['author-name'].str.lower()
    df['matched-name'] = df['matched-name'].str.lower()

    # Apply name matching
    df['lev-ratio'] = df.apply(lambda row: lev.ratio(str(row['author-name']), str(row['matched-name'])), 1)

    df['rank-order'] = df.sort_values(['zeno-order-id-before-review'], ascending=[False]) \
                           .groupby(['review-id', 'matched-name']) \
                           .cumcount() + 1

    top_df = df[(df['rank-order'] == 1)]
    top_df['top-matches'] = top_df.sort_values(['lev-ratio'], ascending=[False]).groupby(['review-id']).cumcount() + 1

    # fetch top 3 name matches
    top_df = top_df[(top_df['top-matches'] <= 3)]
    top_df = top_df.sort_values(['star-rating', 'review-id', 'top-matches'], ascending=[True, True, True])

    # Adding extra ETL columns
    top_df['created-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
    top_df['updated-at'] = datetime.datetime.now(tz=gettz('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')
    top_df['created-by'] = 'etl-automation'
    top_df['updated-by'] = 'etl-automation'

    s3.write_df_to_db(df=top_df[patients_table_info['column_name']], table_name=patients_table, db=rs_db_write,
                      schema=schema)

    if "@" in email_to:
        """ Which means we want to send an email """
        file_name = 'Zeno_playstore.xlsx'
        columns_in_mail = ['review-id', 'review', 'star-rating', 'review-created-at', 'author-name',
                           'matched-name', 'zeno-order-id-before-review', 'patient-id', 'order-created-at']

        file_path = s3.write_df_to_excel(data={'Zeno Playstore': top_df[columns_in_mail]}, file_name=file_name)
        email = Email()
        email.send_email_file(
            subject="Zeno Playstore",
            mail_body='Zeno Playstore',
            to_emails=email_to,
            file_uris=[],
            file_paths=[file_path]
        )
        logger.info(f"Email has been sent successfully to: {email_to}")

    counter += 1

    # """ For testing only """
    # if counter >= 2:
    #     break

NameError: name 'helper' is not defined

In [18]:
# Closing the DB Connection
rs_db.close_connection()
rs_db_write.close_connection()

Redshift DB connection closed successfully.
Redshift DB connection closed successfully.
